Import libraries

In [1]:
import pandas as pd
import plotly.express as px

# Some Exploratory analysis

Read Data

In [2]:
data=pd.read_csv('../data/impression_log.csv',index_col=0)

See a glimpse of the  columns and data

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 00006a06-14e2-47d9-b999-fbeeac67dd2b to 6404d38e-1669-4e0e-b6a4-dab8dc454f45
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   LogEntryTime        100000 non-null  object 
 1   AdvertiserId        100000 non-null  object 
 2   CampaignId          100000 non-null  object 
 3   AdGroupId           100000 non-null  object 
 4   AudienceID          96546 non-null   object 
 5   CreativeId          100000 non-null  object 
 6   AdFormat            100000 non-null  object 
 7   Frequency           100000 non-null  int64  
 8   Site                100000 non-null  object 
 9   FoldPosition        100000 non-null  int64  
 10  Country             100000 non-null  object 
 11  Region              99999 non-null   object 
 12  City                99999 non-null   object 
 13  DeviceType          100000 non-null  int64  
 14  OSFamily            9999

Aggregate on Campaign id to find the total impressions and the total number of engagements. The count gives us the total impressions while the sum gives us the number of impressions.

In [4]:
sum=data.groupby(["CampaignId","Site"],as_index=False).agg("sum")
count=data.groupby(["CampaignId","Site"],as_index=False).agg("count")

Visualization to see the distribution of engagement/impressions with the color showing us the engagement rate. This is to see the distribution of engagmenets and impressions

In [6]:

fig = px.scatter(x=count["engagement"], y=sum["engagement"], color=sum["engagement"]/count["engagement"],labels={
                      "x":"Impressions","y":"Engagement","color":"Engagement Rate"
                    })
fig.show()

See the ratio of engagements. We can see that a majortiy 87K+ of the impressions had no engagment.

In [8]:
counts=data.groupby("engagement").size().reset_index(name="counts")

In [9]:
fig = px.bar(counts,x="engagement",y="counts",labels={
                      "engagement":"Engagement","counts":"Number of Impressions"
                    })
# fig.update_xaxes(type='Engagement')
fig.show()

Seeing the distribution of clicks. This shows that a majority of engagments don't turn into clicks. User click is predicated by engagment as well.

In [12]:
counts=data.groupby(["click","engagement"],as_index=False).size()
fig = px.bar(counts,x="click",y="size",color="engagement",labels={
                      "click":"Clicked Ad","size":"Number of Impressions","engagement":"Impression engaged"
                    })
fig.show()

# Preparing the data for scoring and sorting

Aggregating data for scoring and ranking. 

In [20]:
aggregated_data=data.groupby(["CampaignId","Site"],as_index=False).agg(TotalEngagement=pd.NamedAgg(column='engagement', aggfunc='sum'), Impressions=pd.NamedAgg(column='engagement', aggfunc='count'),TotalClicks=pd.NamedAgg(column='click', aggfunc='sum'))

In [21]:
aggregated_data

,CampaignId,Site,TotalEngagement,Impressions,TotalClicks
0,0k2sw7w,247sports.com,0,1,0
1,0k2sw7w,arealnews.com,0,1,0
2,0k2sw7w,blackdoctor.org,0,1,0
3,0k2sw7w,comicbook.com,0,2,0
4,0k2sw7w,dailyvoice.com,0,2,0
...,...,...,...,...,...
19977,z5cjizq,www.wordplays.com,0,6,0
19978,z5cjizq,www.wral.com,0,2,0
19979,z5cjizq,www.wsbtv.com,0,1,0
19980,z5cjizq,www.yahoo.com,2,78,2


The campaigns we have available in our dataset

In [34]:
aggregated_data["CampaignId"].unique()

array(['0k2sw7w', '5qtwg2a', '758ifo0', '8muucqy', '9n6o7jf', 'awbu4q4',
       'dnpjc6c', 'fiwemi8', 'shg01yl', 't29si1w', 'v0fbymt', 'xygdmea',
       'z5cjizq'], dtype=object)

# Score

The scoring function takes the number of engagements and the total number of impressions and generates a score that is based on Laplace's Rule of Succession. The rule gives us the ability to calculate the probability the next impression is going to be engaged based on the historical data we have. This takes care of the issue where the accuracy of the probability given by just using engagement rate is put into question due to the number of observations. The higher the number of impressions the smaller the deviation of our new probability from the ideal probabilty given by engagement rate. 

In [24]:
def score(row:pd.Series)->float:
    """Scoring function that takes the row containing impressions and total engagement to give a score

    Args:
        row (pd.Series): The website entry that will be scored

    Returns:
        float: The score
    """
    final_score=(row["TotalEngagement"]+1)/(row["Impressions"]+2)
    return final_score

# Sort

Sorting the websites is selecting the websites based on campaign id, applying scores to all of the websites and then using the score to sort the values

In [37]:
def sortWebsites(CampaignId:str,dataframe:pd.DataFrame)->pd.DataFrame:
    """Gives a sorted dataframe of the websites in that campaign based on score

    Args:
        CampaignId (str): The id identifying the campagin
        dataframe (pd.DataFrame): The dataframe for all campagins

    Returns:
        pd.DataFrame: A sorted list of the website in the campagin
    """
    rslt_df = dataframe[dataframe['CampaignId']==CampaignId]
    rslt_df=rslt_df.copy()
    rslt_df["score"]=rslt_df.apply(score, axis=1)
    rslt_df=rslt_df.sort_values("score",ascending=False)
    return rslt_df

An exmaple sorted campaign. We can see that the score has given more credance to those with a higher number of impressions when it comes to the score even though the engagment rate is the same. 

In [39]:
sortWebsites("awbu4q4",aggregated_data)

,CampaignId,Site,TotalEngagement,Impressions,TotalClicks,score
7259,awbu4q4,www.movieinsider.com,3,3,2,0.800000
6347,awbu4q4,wheretheroadforks.com,3,3,0,0.800000
6178,awbu4q4,theyardandgarden.com,2,2,1,0.750000
6608,awbu4q4,www.cosmopolitan.com,2,2,0,0.750000
5721,awbu4q4,pomeranian.org,2,2,0,0.750000
...,...,...,...,...,...,...
7971,awbu4q4,www.wordplays.com,0,47,0,0.020408
7711,awbu4q4,www.thedailybeast.com,1,100,0,0.019608
4673,awbu4q4,belleofthekitchen.com,0,51,0,0.018868
7356,awbu4q4,www.ondemandkorea.com,1,145,0,0.013605


# Other approaches considered/ Process to arrive at final score

<ul><li>One of the approaches considered was the use of significance tests to compare the sites against eachother. Then we can use a sorting algorithm to compare values and to assign them ranks. This doesn't create a score but would give us a proof that one is more performant than the other. This approach was not successful as seen in the Scoring notebook. The p values generated by small changes meant that the cutoff point for our significance level would have to be very high and it still had not mitigated the issue of smaller number of impressions meaning a higher probability. Different significance tests were considered but none failed to satisfy the criteria. </li>
<li>
Another approach considered is similar to the approach above and is based on this <a href="https://www.tandfonline.com/doi/full/10.1080/03610926.2014.953693">paper</a>. This also was not successful due to the process not being definitive and the intervals being not so different. This also wouldn't work for smaller number of impressions due to it being a poisson approximation. 
</li>
<li>
An approach that would work in a live ranking scenario but would not work for this scenario is the use of Multi-Armed Bandit Problem to model our ranking problem. There are many approaches/solutions to the problem but are only effective with feedback loops based on live feedback of engagement. This approach doesn't score the websites but can find us the optimum ranking of the websites to maximize engagement
</li>

</ul>